# Evolver Loop 2 - LB Feedback Analysis

**LB Score**: 70.6151 (matches CV exactly - no gap)
**Target**: 68.882921
**Gap**: 1.73 points (2.5% improvement needed)

## Key Insights from Research

1. **Zaburo's Lattice Approach**: Uses alternating rows at 0° and 180° with row spacing of 1.0 units. Achieves 88.33 as initial solution.

2. **Chistyakov's Backward Propagation**: Removes trees from larger N to improve smaller N. Key insight: only try removing trees that touch the bounding box.

3. **Per-N Score Analysis**: Small N values contribute disproportionately to score.

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os

os.chdir('/home/code')

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_vertices(x, y, angle_deg):
    rad = np.radians(angle_deg)
    cos_a, sin_a = np.cos(rad), np.sin(rad)
    rx = TX * cos_a - TY * sin_a + x
    ry = TX * sin_a + TY * cos_a + y
    return rx, ry

def calculate_score_for_n(trees):
    all_xs = []
    all_ys = []
    for x, y, angle in trees:
        rx, ry = get_tree_vertices(x, y, angle)
        all_xs.extend(rx)
        all_ys.extend(ry)
    width = max(all_xs) - min(all_xs)
    height = max(all_ys) - min(all_ys)
    side = max(width, height)
    n = len(trees)
    return (side ** 2) / n

print('Functions defined')

Functions defined


In [2]:
# Load baseline submission
df = pd.read_csv('experiments/001_valid_baseline/submission.csv')

def parse_submission(df):
    configs = defaultdict(list)
    for _, row in df.iterrows():
        parts = row['id'].split('_')
        n = int(parts[0])
        x = float(str(row['x']).replace('s', ''))
        y = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        configs[n].append((x, y, deg))
    return dict(configs)

configs = parse_submission(df)
print(f'Loaded {len(configs)} N values')

Loaded 200 N values


In [3]:
# Calculate per-N scores
scores_by_n = {}
for n in range(1, 201):
    trees = configs[n]
    scores_by_n[n] = calculate_score_for_n(trees)

total = sum(scores_by_n.values())
print(f'Total score: {total:.6f}')
print(f'Target: 68.882921')
print(f'Gap: {total - 68.882921:.6f}')

Total score: 70.615102
Target: 68.882921
Gap: 1.732181


In [4]:
# Analyze score contributions by N range
ranges = [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]

print('\nScore contribution by N range:')
print('-' * 50)
for start, end in ranges:
    range_score = sum(scores_by_n[n] for n in range(start, end+1))
    pct = 100 * range_score / total
    print(f'N={start:3d}-{end:3d}: {range_score:8.4f} ({pct:5.1f}%)')


Score contribution by N range:
--------------------------------------------------
N=  1- 10:   4.3291 (  6.1%)
N= 11- 20:   3.7240 (  5.3%)
N= 21- 50:  10.9795 ( 15.5%)
N= 51-100:  17.6063 ( 24.9%)
N=101-150:  17.1341 ( 24.3%)
N=151-200:  16.8421 ( 23.9%)


In [5]:
# Top 20 worst N values (highest score contribution)
print('\nTop 20 worst N values (highest score contribution):')
print('-' * 50)
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    print(f'N={n:3d}: {score:.6f}')


Top 20 worst N values (highest score contribution):
--------------------------------------------------
N=  1: 0.661250
N=  2: 0.450779
N=  3: 0.434745
N=  5: 0.416850
N=  4: 0.416545
N=  7: 0.399897
N=  6: 0.399610
N=  9: 0.387415
N=  8: 0.385407
N= 15: 0.376949
N= 10: 0.376630
N= 21: 0.376451
N= 20: 0.376057
N= 22: 0.375258
N= 11: 0.374924
N= 16: 0.374128
N= 26: 0.373997
N= 12: 0.372724
N= 13: 0.372294
N= 25: 0.372144


In [6]:
# Calculate theoretical minimum for N=1 (single tree at 45°)
# At 45°, the bounding box is minimized
import math

# Tree dimensions at 0°: width=0.7, height=1.0 (from -0.2 to 0.8)
# At 45°, the bounding box diagonal becomes the side
# Theoretical minimum for N=1
angle_45 = 45
rx, ry = get_tree_vertices(0, 0, angle_45)
width = max(rx) - min(rx)
height = max(ry) - min(ry)
side = max(width, height)
theoretical_n1 = side ** 2

print(f'\nN=1 Analysis:')
print(f'  Current score: {scores_by_n[1]:.6f}')
print(f'  Theoretical at 45°: {theoretical_n1:.6f}')
print(f'  Difference: {scores_by_n[1] - theoretical_n1:.6f}')


N=1 Analysis:
  Current score: 0.661250
  Theoretical at 45°: 0.661250
  Difference: -0.000000


In [7]:
# Test Zaburo's lattice approach for N=10
def zaburo_lattice(n):
    """Generate lattice solution like Zaburo's kernel."""
    best_score = float('inf')
    best_trees = None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            all_trees = []
            rest = n
            r = 0
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else 0.35  # 0.7/2
                y = r // 2 * 1.0 if r % 2 == 0 else (0.8 + (r - 1) // 2 * 1.0)
                
                for i in range(m):
                    x = 0.7 * i + x_offset
                    all_trees.append((x, y, angle))
                
                r += 1
            
            score = calculate_score_for_n(all_trees)
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

# Test on N=10, 20, 50
print('\nZaburo Lattice vs Baseline:')
print('-' * 50)
for n in [10, 20, 50, 100, 200]:
    lattice_score, _ = zaburo_lattice(n)
    baseline_score = scores_by_n[n]
    diff = baseline_score - lattice_score
    better = '✓ BETTER' if diff > 0 else '✗ worse'
    print(f'N={n:3d}: Lattice={lattice_score:.6f}, Baseline={baseline_score:.6f}, Diff={diff:+.6f} {better}')


Zaburo Lattice vs Baseline:
--------------------------------------------------
N= 10: Lattice=0.484000, Baseline=0.376630, Diff=-0.107370 ✗ worse
N= 20: Lattice=0.496125, Baseline=0.376057, Diff=-0.120068 ✗ worse


N= 50: Lattice=0.480200, Baseline=0.360753, Diff=-0.119447 ✗ worse


N=100: Lattice=0.396900, Baseline=0.343395, Diff=-0.053505 ✗ worse


N=200: Lattice=0.405000, Baseline=0.337549, Diff=-0.067451 ✗ worse


In [8]:
# Key insight: The baseline is already highly optimized
# The lattice approach is WORSE than baseline for all N
# This confirms that the baseline uses more sophisticated optimization

print('\n' + '='*60)
print('KEY INSIGHTS:')
print('='*60)
print('1. Baseline is already highly optimized (better than simple lattice)')
print('2. N=1 is already optimal at 0.6612')
print('3. Small N (1-10) contribute 6.1% of total score')
print('4. Need to find improvements in specific N values')
print('5. Backward propagation (Chistyakov) might help for some N')


KEY INSIGHTS:
1. Baseline is already highly optimized (better than simple lattice)
2. N=1 is already optimal at 0.6612
3. Small N (1-10) contribute 6.1% of total score
4. Need to find improvements in specific N values
5. Backward propagation (Chistyakov) might help for some N
